# Finding the best model

After we have processed our dataset, we need to find a model to predict new values. We will explore different models to see which ones perform better in this particular dataset.

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from distributed import Client
import dask.dataframe as dd
from dask import compute
from sklearn import set_config
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
# from dask_ml.preprocessing import Categorizer, StandardScaler, DummyEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
# from dask_ml.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV
# from dask_ml.model_selection import GridSearchCV

# from dask_ml.model_selection import train_test_split
from sklearn.model_selection import train_test_split


In [ ]:

# client = Client(n_workers=2, threads_per_worker=2, memory_limit='4GB')
ddf = dd.read_parquet('/home/diego/Coding/code-challenge-2020/data_root/processed/train.parquet', engine='pyarrow')
# X, y = ddf.drop(['points'], axis=1), ddf['points']
X, y = compute(ddf.drop(['points'], axis=1), ddf[['points']]) # using pandas dataframes only. Dask gives more issues...
X.head()


In [ ]:
set_config(display='diagram')  # Allows us to visualize pipeline
num_proc = make_pipeline(StandardScaler())
cat_proc = make_pipeline(OneHotEncoder(handle_unknown='ignore'))
numerical_columns = ['price']
categorical_columns = X.columns.drop('price').to_list()
preprocessor = make_column_transformer((num_proc, numerical_columns),
                                       (cat_proc, categorical_columns))
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=0, shuffle=True)
score_matrix = pd.DataFrame(columns=['model', 'R2', 'MSE', 'MAE']).set_index('model')

X.describe(include="all")

## Baseline - Linear Regression

First we will create a baseline using a simple linear regression model.

In [ ]:
linear_pipeline = make_pipeline(preprocessor, LinearRegression())
linear_pipeline

In [ ]:
linear_pipeline.fit(X_train, y_train)
d = {
    'R2': [r2_score(y_test, linear_pipeline.predict(X_test))],
    'MSE': [mean_squared_error(y_test, linear_pipeline.predict(X_test))],
    'MAE': [mean_absolute_error(y_test, linear_pipeline.predict(X_test))]
}

score_matrix = pd.concat([score_matrix, pd.DataFrame(d, index=['linear'])])
score_matrix


## Ridge regression

In [ ]:
# Ridge regression using 5-fold cross validation to find best parameters
ridge_pipeline = make_pipeline(preprocessor, RidgeCV(alphas=np.logspace(-2, 1, 200), cv=5)) 
ridge_pipeline.fit(X_train, y_train)
d = {
    'R2': [r2_score(y_test, ridge_pipeline.predict(X_test))],
    'MSE': [mean_squared_error(y_test, ridge_pipeline.predict(X_test))],
    'MAE': [mean_absolute_error(y_test, ridge_pipeline.predict(X_test))]
}
print(f"Best alpha: {ridge_pipeline.named_steps['ridgecv'].alpha_}")
score_matrix = pd.concat([score_matrix, pd.DataFrame(d, index=['ridge'])])
score_matrix


We see that with some regularization (Ridge regression), the model starts to get better. Let's take a closer look at the results.

In [ ]:
feature_names = (ridge_pipeline.named_steps['columntransformer']
                 .named_transformers_['pipeline-2'].named_steps['onehotencoder']
                 .get_feature_names(input_features=categorical_columns))

feature_names = np.concatenate([numerical_columns, feature_names])

coefs = pd.DataFrame(
    ridge_pipeline.named_steps['ridgecv'].coef_.transpose(),
    columns=['Coefficients'], index=feature_names
)
ordered_coefs = coefs.loc[coefs.abs().sort_values(by='Coefficients', ascending=False).index]
price_idx = ordered_coefs.reset_index()
price_idx = price_idx.loc[price_idx['index']=='price']
print(f"Price is in the position {price_idx.index[0]} out of {len(coefs)}")
ordered_coefs = ordered_coefs.iloc[0:30]
ordered_coefs.plot(kind='barh', figsize=(9, 7))
plt.title('Ridge model')
plt.axvline(x=0, color='.5')
plt.subplots_adjust(left=.3)

We plot the 30 bigger coefficients of the model. They tend to be associated with the features 'winery' or 'region_1'. We can further inspect the model using [permutation feature importance](https://scikit-learn.org/stable/modules/permutation_importance.html). This gives us an intuition of which features are most relevant when making predictions. This data confirms our intuition that features like price, region or even the wine's reviewer are more relevant than the variety of the grape for example.

In [ ]:
# Feature importance
from sklearn.inspection import permutation_importance
r = permutation_importance(ridge_pipeline, X_test, y_test,
                           n_repeats=30,
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X.columns[i]:<14}"
              f"{r.importances_mean[i]:.3f}"
              f" ± {r.importances_std[i]:.3f}")


We can also plot the predicted values againts the real targets. A perfect predictor should produce points that fall along the red line.

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
plt.scatter(y_test, ridge_pipeline.predict(X_test))
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c="red")
# plt.text(3, 20, string_score)
plt.title('Ridge model')
plt.ylabel('Model predictions')
plt.xlabel('Truths')

## Support vector machine

In [ ]:
# svr_pipeline = make_pipeline(preprocessor, SVR(kernel='rbf', C=10, gamma=0.0167)) # Best regressor
# svr_pipeline.fit(X_train, y_train.values.ravel())

# d = {
#     'R2': [r2_score(y_test, svr_pipeline.predict(X_test))],
#     'MSE': [mean_squared_error(y_test, svr_pipeline.predict(X_test))],
#     'MAE': [mean_absolute_error(y_test, svr_pipeline.predict(X_test))]
# }

# score_matrix = pd.concat([score_matrix, pd.DataFrame(d, index=['svr_rbf'])])
# score_matrix

In [ ]:
svr_pipeline = make_pipeline(preprocessor, SVR())
param_grid = [
    {
        'svr__kernel': ['rbf'],
        'svr__gamma': np.logspace(-3, 1, 10), 
        'svr__C': [1, 10, 100]
    },
]
grid_search = GridSearchCV(svr_pipeline, param_grid)
grid_search.fit(X_train, y_train.values.ravel())
d = {
    'R2': [r2_score(y_test, grid_search.predict(X_test))],
    'MSE': [mean_squared_error(y_test, grid_search.predict(X_test))],
    'MAE': [mean_absolute_error(y_test, grid_search.predict(X_test))]
}
print(f"Best parameter set is: {grid_search.best_params_}")
score_matrix = pd.concat([score_matrix, pd.DataFrame(d, index=['svr_rbf'])])
score_matrix


In [ ]:
# Feature importance
from sklearn.inspection import permutation_importance
r = permutation_importance(grid_search, X_test, y_test,
                           n_repeats=30,
                           random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{X.columns[i]:<14}"
              f"{r.importances_mean[i]:.3f}"
              f" ± {r.importances_std[i]:.3f}")
fig, ax = plt.subplots(figsize=(5, 5))
plt.scatter(y_test, ridge_pipeline.predict(X_test))
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c="red")
# plt.text(3, 20, string_score)
plt.title('Ridge model')
plt.ylabel('Model predictions')
plt.xlabel('Truths')

## KNN regression

In [ ]:
knn_pipeline = make_pipeline(preprocessor, KNeighborsRegressor())
param_grid = [
    {
        'kneighborsregressor__weights': ['uniform','distance'],
        'kneighborsregressor__n_neighbors': np.arange(1,20,1),
        'kneighborsregressor__p': [1, 2]
    }
]
grid_search = GridSearchCV(knn_pipeline, param_grid)
grid_search.fit(X_train, y_train)
d = {
    'R2': [r2_score(y_test, grid_search.predict(X_test))],
    'MSE': [mean_squared_error(y_test, grid_search.predict(X_test))],
    'MAE': [mean_absolute_error(y_test, grid_search.predict(X_test))]
}
print(f"Best parameter set is: {grid_search.best_params_}")
score_matrix = pd.concat([score_matrix, pd.DataFrame(d, index=['knn'])])
score_matrix


## Final thoughts

If we take a look at the score matrix, we see that the Support vector machine with radial kernel outperforms the rest in all three metrics. This could imply that the data follows a distribution that can't be described using linear operations.